In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [10]:
x_train, y_train,x_test_original, y_test_original = ReadTS('Strawberry')
y_train = y_train - 1
y_test_original = y_test_original - 1
print(x_train.shape)
#print(y_train)
print(x_train[1])

(613, 235, 1)
[[-3.3629864e-01]
 [-3.3609084e-01]
 [-3.3338936e-01]
 [-3.1136196e-01]
 [-2.6086516e-01]
 [-1.9582197e-01]
 [-1.7524920e-01]
 [-1.9810783e-01]
 [-2.2720064e-01]
 [-2.4839682e-01]
 [-2.6460567e-01]
 [-2.7977549e-01]
 [-2.8995797e-01]
 [-2.9307505e-01]
 [-2.7603498e-01]
 [-2.1722596e-01]
 [-9.4204969e-02]
 [ 6.7260082e-02]
 [ 1.5412288e-01]
 [ 1.7698151e-01]
 [ 2.8109218e-01]
 [ 4.1865957e-01]
 [ 4.6375341e-01]
 [ 4.8765107e-01]
 [ 5.5144743e-01]
 [ 5.9841153e-01]
 [ 6.3228386e-01]
 [ 7.0273000e-01]
 [ 8.3385542e-01]
 [ 1.0294006e+00]
 [ 1.2234912e+00]
 [ 1.3448497e+00]
 [ 1.4618443e+00]
 [ 1.6702735e+00]
 [ 1.9468629e+00]
 [ 2.1746180e+00]
 [ 2.2884955e+00]
 [ 2.3564480e+00]
 [ 2.4728192e+00]
 [ 2.6866513e+00]
 [ 3.0056331e+00]
 [ 3.3503828e+00]
 [ 3.5721115e+00]
 [ 3.5696178e+00]
 [ 3.3489282e+00]
 [ 3.0515582e+00]
 [ 2.8753389e+00]
 [ 2.8522725e+00]
 [ 2.7718516e+00]
 [ 2.4626367e+00]
 [ 2.0632341e+00]
 [ 1.8176077e+00]
 [ 1.7835276e+00]
 [ 1.8070096e+00]
 [ 1.7037302e+

In [11]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    # 获取原始数据集的形状
    original_shape = TS.shape
    
    # 将数据集reshape成二维数组
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    # 初始化StandardScaler
    scaler = StandardScaler()
    
    # 对数据集进行标准化
    TS_normalized = scaler.fit_transform(TS_2d)
    
    # 将标准化后的数据集reshape回原始形状
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
print(x_train[1])

[[ 0.86259263]
 [ 0.83121268]
 [ 0.78001627]
 [ 0.73169982]
 [ 0.69587762]
 [ 0.68355889]
 [ 0.58394953]
 [ 0.46529293]
 [ 0.42840521]
 [ 0.46930512]
 [ 0.52806311]
 [ 0.5558983 ]
 [ 0.5819564 ]
 [ 0.6337927 ]
 [ 0.74085757]
 [ 0.95217094]
 [ 1.30524788]
 [ 1.65862542]
 [ 1.7103277 ]
 [ 1.43770521]
 [ 1.21438452]
 [ 0.91633142]
 [ 0.31927703]
 [-0.1686687 ]
 [-0.38119617]
 [-0.41867934]
 [-0.40251721]
 [-0.4234195 ]
 [-0.53264268]
 [-0.7853699 ]
 [-1.16983387]
 [-1.52135747]
 [-1.66644558]
 [-1.59294303]
 [-1.4226192 ]
 [-1.2319741 ]
 [-1.10458868]
 [-0.92318702]
 [-0.45875399]
 [ 0.16262324]
 [ 0.78608956]
 [ 1.48281803]
 [ 2.13450254]
 [ 2.43836583]
 [ 2.33694639]
 [ 1.83560968]
 [ 1.13833947]
 [ 0.93151161]
 [ 1.1539546 ]
 [ 1.40647917]
 [ 1.47098044]
 [ 1.32172882]
 [ 1.06200082]
 [ 0.76207482]
 [ 0.3225002 ]
 [-0.32938776]
 [-1.07978489]
 [-1.37983989]
 [-1.06244777]
 [-0.63994097]
 [-0.33410837]
 [-0.1311355 ]
 [ 0.00956373]
 [ 0.15831916]
 [ 0.46260036]
 [ 1.07743287]
 [ 1.52600

In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 2)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-11-29 14:26:16.722932: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


16/16 [==============================] - 1s 22ms/step - loss: 0.5814 - sparse_categorical_accuracy: 0.6612 - val_loss: 0.6717 - val_sparse_categorical_accuracy: 0.5528
Epoch 2/100
16/16 [==============================] - 0s 14ms/step - loss: 0.4695 - sparse_categorical_accuracy: 0.7551 - val_loss: 0.6491 - val_sparse_categorical_accuracy: 0.7805
Epoch 3/100
16/16 [==============================] - 0s 13ms/step - loss: 0.4703 - sparse_categorical_accuracy: 0.7592 - val_loss: 0.6609 - val_sparse_categorical_accuracy: 0.5935
Epoch 4/100
16/16 [==============================] - 0s 13ms/step - loss: 0.4490 - sparse_categorical_accuracy: 0.7776 - val_loss: 0.6796 - val_sparse_categorical_accuracy: 0.5366
Epoch 5/100
16/16 [==============================] - 0s 13ms/step - loss: 0.4603 - sparse_categorical_accuracy: 0.7490 - val_loss: 0.7008 - val_sparse_categorical_accuracy: 0.4715
Epoch 6/100
16/16 [==============================] - 0s 13ms/step - loss: 0.4334 - sparse_categorical_accuracy: 

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

12/12 [==============================] - 0s 6ms/step - loss: 0.5520 - sparse_categorical_accuracy: 0.7432
Test accuracy 0.7432432174682617
Test loss 0.5519823431968689


### Create perturbed test sets and calculate decrease in accuracy

In [7]:
print(y_test_original.shape)

(370,)


In [8]:
print(inspect.getsource(perturb))
print(inspect.signature(perturb))

def perturb(perturbation_strategy, ts, index0, index1, global_ts = [],bg = []):
    if perturbation_strategy == 'RBP':
        return RBPIndividual(ts, index0, index1)
    if perturbation_strategy == 'zero':
        return zeroPerturb(ts, index0, index1)
    if perturbation_strategy == 'noise':
        return noisePerturb(ts, index0, index1)
    if perturbation_strategy == 'blur':
        return blurPerturb(ts, index0, index1)
    if perturbation_strategy == 'RBP1':
        return RBPIndividualNew1(global_ts, ts, index0, index1) 
    if perturbation_strategy == 'RBP1fast':
        return RBPIndividualNew1fast(bg, ts, index0, index1)  
    if perturbation_strategy == 'RBP2':
        return RBPIndividualNew2(global_ts, ts, index0, index1)    

(perturbation_strategy, ts, index0, index1, global_ts=[], bg=[])


In [65]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 90
index_1 = 180
x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

bg = multisamplebackgroundIdentification(perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1fast',x,index_0,index_1, bg = bg) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

#x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
#x_test_noise = np.expand_dims(x_test_noise, axis=-1)
#predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

12/12 [==============================] - 0s 4ms/step


In [66]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
#l2norm(x_test_noise)
l2norm(x_test_blur)

5.364917292112747
8.022776592677545
7.794814037319751
0.289085299586043


In [67]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [68]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
#noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [69]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    #" Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.7432432432432432
 RBP Accuracy: 0.7351351351351352
 RBP1 Accuracy: 0.6945945945945946
 Zero Accuracy: 0.7297297297297297
 Blur Accuracy: 0.7432432432432432

